<a href="https://colab.research.google.com/github/mitprojects/R_System/blob/main/Recommendation_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 19.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406398 sha256=13acd80f47e9e7937ac67e7beeb3b0e3dd7f3e63ced0ef93d07e0c71ebf5b3de
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
import pandas as pd
import scipy.sparse as sparse
import os
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [4]:
Data_dir = os.getcwd() + "/drive/MyDrive/Colab Notebooks/"

In [5]:
new_data = pd.read_csv(Data_dir+'new Data/archive/new_RS_data.csv')
new_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [7]:
new_data.rename(columns={'content_id':'product_id','contentId':'productId'},inplace=True)

In [8]:
new_data.head()

,product_name,product_id,personId,productId,eventStrength,person_id
0,XPS 13 2-in-1 Laptop,1,-9207251133131336884,-9216926795620865886,3.0,0
1,XPS 17 Laptop,3,-9016528795238256703,-9192549002213406534,1.0,1
2,Inspiron 14 Laptop,7,-9016528795238256703,-9176143510534135851,1.0,1
3,Inspiron 15 3000 Laptop,3,-9016528795238256703,-9192549002213406534,1.0,1
4,Alienware m17 R4 Gaming Laptop,12,-9009798162809551896,-9160910454530522563,1.0,2


In [9]:
sparse_content_person = sparse.csr_matrix((new_data['eventStrength'].astype(float), (new_data['product_id'], new_data['person_id'])))
sparse_person_content = sparse.csr_matrix((new_data['eventStrength'].astype(float), (new_data['person_id'], new_data['product_id'])))

In [10]:
sparse_content_person

<13x128 sparse matrix of type '<class 'numpy.float64'>'
	with 157 stored elements in Compressed Sparse Row format>

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


In [12]:
alpha = 15
data = (sparse_content_person * alpha).astype('double')

# Fit the model
model.fit(data)

In [13]:
content_id = 5
n_similar = 3

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))

scores = content_vecs.dot(content_vecs[content_id]) / content_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])

In [14]:
for content in similar:
    idx, score = content
    print(new_data.product_name.loc[new_data.product_id == idx].iloc[0])

Alienware m15 R4 Gaming Laptop
XPS 13 2-in-1 Laptop
Inspiron 14 Laptop


In [18]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()
    # Add 1 to everything, so that product with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make product already interacted zero
    person_interactions[person_interactions > 1] = 0
    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled
    # Sort the indices of the content into order of best recommendations
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store product_name and scores
    product_name = []
    scores = []

    for idx in content_idx:
        # Append product name and scores to the list
        product_name.append(new_data.product_name.loc[new_data.product_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'product_name': product_name, 'score': scores})

    return recommendations

In [19]:
# Get the trained person and content vectors. We convert them to csr matrices
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)



In [20]:
# Create recommendations for person with id 8
person_id = 8

recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)

print(recommendations)

                     product_name     score
0  Alienware m15 R3 Gaming Laptop  0.054579
1  Alienware m15 R4 Gaming Laptop  0.031718
2  Alienware m17 R3 Gaming Laptop  0.031196
3              Inspiron 14 Laptop  0.030415
4         New XPS 15 Touch Laptop  0.028810
5              Inspiron 17 Laptop  0.028357
6         Inspiron 14 3000 Laptop  0.026784
7            XPS 13 2-in-1 Laptop  0.026514
8                   XPS 15 Laptop  0.022814
9  Alienware m17 R4 Gaming Laptop  0.000000
